In [1]:
import pandas as pd
import numpy as np
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch_geometric.nn import GCNConv
from torch_geometric.loader import DataLoader
from torch_geometric.data import Data
from torch_geometric.nn import SAGEConv

import re
import nltk
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.utils import resample

from keras.utils import pad_sequences

from gensim.models import Word2Vec
from eunjeon import Mecab

# mecab 모델 다운로드
mecab = Mecab()

In [2]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
data_dir = '../analysis_files/files/'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
# 감성대화말뭉치(최종데이터)_Training.csv 파일을 pandas로 읽어옵니다.
df = pd.read_csv(os.path.join(data_dir, '감성대화말뭉치(최종데이터)_Training.csv'), encoding='cp949')
df = df[['사람문장1', '시스템문장1', '사람문장2','시스템문장2','사람문장3','시스템문장3','감정_대분류', '상황키워드']]
df

,사람문장1,시스템문장1,사람문장2,시스템문장2,사람문장3,시스템문장3,감정_대분류,상황키워드
0,일은 왜 해도 해도 끝이 없을까? 화가 난다.,많이 힘드시겠어요. 주위에 의논할 상대가 있나요?,그냥 내가 해결하는 게 나아. 남들한테 부담 주고 싶지도 않고.,혼자 해결하기로 했군요. 혼자서 해결하기 힘들면 주위에 의논할 사람을 찾아보세요.,NaN,NaN,분노,"진로,취업,직장"
1,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.,급여가 줄어 속상하시겠어요. 월급이 줄어든 것을 어떻게 보완하실 건가요?,최대한 지출을 억제해야겠어. 월급이 줄어들었으니 고정지출을 줄일 수밖에 없을 것 같아.,월급이 줄어든 만큼 소비를 줄일 계획이군요.,NaN,NaN,분노,"진로,취업,직장"
2,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,회사 동료 때문에 스트레스를 많이 받는 것 같아요. 문제 해결을 위해 어떤 노력을 ...,잘 안 맞는 사람이랑 억지로 잘 지내는 것보단 조금은 거리를 두고 예의를 갖춰서 대...,스트레스받지 않기 위해선 인간관계에 있어 약간의 거리를 두는 게 좋겠군요.,NaN,NaN,분노,"진로,취업,직장"
3,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...,관련 없는 심부름을 모두 하게 되어서 노여우시군요. 어떤 것이 상황을 나아질 수 있...,직장 사람들과 솔직하게 이야기해보고 싶어. 일하는 데에 방해된다고.,직장 사람들과 이야기를 해 보겠다고 결심하셨군요.,NaN,NaN,분노,"진로,취업,직장"
4,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.,무시하는 것 같은 태도에 화가 나셨군요. 상대방의 어떤 행동이 그런 감정을 유발하는...,상사인 나에게 먼저 인사하지 않아서 매일 내가 먼저 인사한다고!,항상 먼저 인사하게 되어 화가 나셨군요. 어떻게 하면 신입사원에게 화났음을 표현할 ...,NaN,NaN,분노,"진로,취업,직장"
...,...,...,...,...,...,...,...,...
51625,나이가 먹고 이제 돈도 못 벌어 오니까 어떻게 살아가야 할지 막막해. 능력도 없고.,경제적인 문제 때문에 막막하시군요. 마음이 편치 않으시겠어요.,아무것도 할 수 없는 내가 무가치하게 느껴지고 실망스러워.,지금 할 수 있는 가장 합리적인 행동은 무엇인가요?,노년층을 위한 경제적 지원이나 부업 같은 것도 알아보아야겠어.,좋은 결과 얻으시길 바랄게요.,분노,재정
51626,몸이 많이 약해졌나 봐. 이제 전과 같이 일하지 못할 것 같아 너무 짜증 나.,건강에 대한 어려움 때문에 기분이 좋지 않으시군요. 속상하시겠어요.,마음 같아서는 다 할 수 있는 일인데 이젠 몸이 안 따라와 주니 화만 나.,어떻게 하면 지금의 기분을 나아지게 할 수 있을까요?,남편과 함께 게이트볼이나 치러 가야겠어. 그럼 기분이 나아질 것 같아.,남편과 함께하는 좋은 외출 시간 되시길 바랄게요.,불안,재정
51627,이제 어떻게 해야 할지 모르겠어. 남편도 그렇고 노후 준비도 안 되어서 미래가 걱정돼.,노후 준비에 대한 어려움 때문에 걱정이 많으시겠어요.,주변 사람들은 다 노후 준비도 잘해두었던데 난 어떻게 해야 할지 모르겠어. 막막하기...,지금의 상황에서 할 수 있는 가장 좋은 행동이 무엇일까요?,남편과 함께 실버 일자리나 노년층을 위한 국가 지원에 대해 자세히 알아보아야겠어.,좋은 정보 많이 얻으셔서 걱정을 좀 덜으셨으면 좋겠어요.,상처,재정
51628,몇십 년을 함께 살았던 남편과 이혼했어. 그동안의 세월에 배신감을 느끼고 너무 화가 나.,가족과의 문제 때문에 속상하시겠어요.,이제 할 수 있는 일도 없고 이렇게 힘들게 사는 게 불만스럽기만 해.,지금의 감정을 나아지게 할 수 있는 어떤 방법이 있을까요?,함께 친하게 지내던 동네 언니 동생들과 빈자리를 조금이나마 채울까 해.,지인분들과 좋은 시간 보내셨으면 좋겠어요.,불안,대인관계


In [5]:
# 문장 특징 추출 함수
def extract_sentence_features(sentence):
    # TF-IDF 벡터화 객체 생성
    
    tfidf_vectorizer = TfidfVectorizer(token_pattern=r"(?u)\b\w+\b")
    
    # 문장 길이
    sentence_length = len(sentence)

    # 품사 개수
    tokens = mecab.pos(sentence)
    pos_tags = [tag for _, tag in tokens]
    num_pos_tags = len(pos_tags)
    
    # 명사 추출
    nouns = mecab.nouns(sentence)
    # TF-IDF 벡터화 및 상위 1개 단어 추출
    if nouns:
        tfidf_matrix = tfidf_vectorizer.fit_transform(nouns)
        feature_names = tfidf_vectorizer.get_feature_names_out()
        tfidf_scores = tfidf_matrix.toarray()[0]
        top_index = np.argmax(tfidf_scores)
        top_word = feature_names[top_index]
        top_score = tfidf_scores[top_index]
        
    else:
        top_word = ''
        top_score = 0.0
        
    return sentence_length, num_pos_tags, top_word, top_score, top_index
        

In [6]:
def word_embedding(sentences, embedding_size=100, window=5, min_count=1, workers=1):
    # Tokenize sentences
    tokenized_sentences = mecab.nouns(sentences)
    # Train Word2Vec model with skip-gram
    model = Word2Vec(tokenized_sentences, vector_size=embedding_size, window=window, min_count=min_count,
                    workers=workers, sg=1)
    embeddings = []
    for sentence in tokenized_sentences:
        # Calculate the average word embedding for the sentence
        sentence_embedding = np.mean([model.wv[word] for word in sentence if word in model.wv], axis=0)
        embeddings.append(sentence_embedding)
    return embeddings

In [7]:
def preprocessing(sentences) :
    sentences = [re.sub(r'\([^)]*\)', '', text) for text in sentences]
    sentences = [text.replace('.', '') for text in sentences]
    sentences = [re.sub(r'[^가-힣\s]', '', text) for text in sentences]
    sentences = [re.sub(r'\b(?:cm|km|etc)\b', '', text) for text in sentences]
    sentences = [text for text in sentences if text != '']
    return sentences

In [ ]:
# 클래스 불균형 처리 함수
# def handle_imbalance(features, labels):
#     print(len(features), len(labels))
#     unique_labels, counts = np.unique(labels, return_counts=True)
#     majority_class = unique_labels[np.argmax(counts)]
#     indices_majority = np.where(labels == majority_class)[0]
#     minority_classes = unique_labels[np.argmin(counts)]
#     indices_minority = np.where(labels == minority_classes)[0]
#     upsampled_indices_minority = resample(indices_minority, replace=True, n_samples=len(indices_majority))
#     upsampled_indices = np.concatenate((indices_majority, upsampled_indices_minority))
#     print(len(upsampled_indices))
#     return features[upsampled_indices], labels[upsampled_indices]

In [8]:
# 문장 특징 및 감정, 상황 데이터 추출
features = []
labels_emotion = []
labels_situation = []
all_sentences = []

for i, row in df.iterrows():
    sentences = [row['사람문장1'], row['사람문장2'], row['사람문장3']]
    sentences = [sentence for sentence in sentences if pd.notnull(sentence)]  # Remove NaN sentences
    sentences = preprocessing(sentences)
    combined_sentence = ' '.join(sentences)  # Combine sentences into a single string
    all_sentences.append(combined_sentence)
        
    if all_sentences:
        sentence_lengths = []
        num_pos_tags = []
        top_words = []
        top_scores = []
        
        length, pos_tags, word, score, index = extract_sentence_features(all_sentences[i])
        embeddings = word_embedding(all_sentences[i])
        sentence_lengths.append(length)
        num_pos_tags.append(pos_tags) 
        top_words.append(word)
        top_scores.append(score)        
        # Add features and labels to respective lists
        features.append([sentence_lengths, num_pos_tags, list(embeddings[index])])
        labels_emotion.append(row['감정_대분류'])
        labels_situation.append(row['상황키워드'])
        
# Convert lists to numpy arrays
features = np.array(features)
labels_emotion = np.array(labels_emotion)
labels_situation = np.array(labels_situation)

C:\Users\User\AppData\Local\Temp\ipykernel_17000\3594693482.py:32: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  features = np.array(features)


In [ ]:
# 클래스 불균형 처리 적용
# features, labels_emotion = handle_imbalance(features, labels_emotion)
# features, labels_situation = handle_imbalance(features, labels_situation)

In [9]:
print(features[3])
print(labels_emotion)
print(labels_situation)

[list([84]) list([43])
 list([-0.0019329969, -0.0008303967, -2.0312611e-05, 0.0018431934, 0.0030380301, 0.0015223678, -0.004299591, 0.0045670033, -0.0037549052, 0.0049284715, -0.0045128227, -0.0069083353, -0.0010893671, 0.0053198114, -0.002600691, -0.000767077, -0.0038087547, 0.005222906, -0.0023366045, -0.003739938, -0.0005245705, 0.002649447, -0.0023031712, -0.0018951987, 0.0006954998, 0.0011950847, 0.0044746697, 0.0012975385, -0.0036351944, -0.0019698527, 0.0010303817, -0.006209628, -0.0010877603, -0.00028728694, -0.0038881775, 0.00379577, 0.004355122, -0.0023120644, -0.005289029, -0.00021185447, 0.0006853882, -0.003428922, -0.0048128306, 0.006918715, 0.0049697724, 0.004914116, -0.004678775, -0.0059743263, 0.0017535919, -0.0023060718, -0.0028964009, -0.0044993986, 0.001105986, 0.0014266217, -0.0032926106, 0.0009970994, 0.0002565342, -0.005492188, -0.0013172854, 0.0012236162, 0.0036492571, -0.0010210552, 0.0017265994, 0.001217567, 0.0001789118, 0.0024527628, -0.0014692013, 0.00746249

In [10]:
# 데이터 분할
X_train, X_test, y_emotion_train, y_emotion_test, y_situation_train, y_situation_test = train_test_split(features, labels_emotion, labels_situation, test_size=0.2, random_state=42)


In [11]:
# 레이블을 정수형으로 변환
label_mapping_emotion = {'기쁨': 0, '당황': 1, '분노': 2, '불안' : 3, '상처' : 4,'슬픔' : 5}  # 감정에 해당하는 레이블과 정수 매핑
label_mapping_situation = {'가족관계': 0, '건강': 1, '건강,죽음': 2, '대인관계' : 3, '대인관계(부부, 자녀)' : 4, '연애,결혼,출산' : 5, '재정' : 6, '재정,은퇴,노후준비' : 7, '직장, 업무 스트레스' : 8, '진로,취업,직장' : 9, '학교폭력/따돌림' : 10, '학업 및 진로' : 11}  # 상황에 해당하는 레이블과 정수 매핑
y_emotion_train = [label_mapping_emotion[label] for label in y_emotion_train]
y_emotion_test = [label_mapping_emotion[label] for label in y_emotion_test]
y_situation_train = [label_mapping_situation[label] for label in y_situation_train]
y_situation_test = [label_mapping_situation[label] for label in y_situation_test]


In [12]:
print(X_train)
print(X_test)
print(y_emotion_train)
print(y_emotion_test)
print(y_situation_train)
print(y_situation_test)

[[list([99]) list([49])
  list([-0.007690846, 0.005270192, -0.0036871887, -0.0021059471, 0.004161467, 0.0008689042, 0.0019706485, 0.0045213485, 0.0009775171, -0.00014273007, 0.0015376539, 0.004659512, -0.0021269263, -0.0021035185, 0.0017414633, -0.0031989263, 0.0028581012, 0.00014660321, -0.0011655085, 0.00092495466, 0.0012222617, -0.0053471434, 0.008661702, 0.009375572, -0.006258187, 0.0016527259, 0.0034516717, 0.0046751704, -0.003291646, 0.0005074787, 0.0037715267, -0.0007945207, -0.0030077866, -0.005705514, 0.006203132, 0.0012772775, 0.0031333878, -0.0012410744, -0.0050876485, 0.0055878335, 0.002605888, -0.00083139143, -0.0017212338, 0.003118161, 0.0067603784, -0.0020413282, 0.0031372434, -0.0010871529, 0.0008127482, -0.004745042, 0.0041516954, 0.002211746, 0.00018654577, -0.0070696417, -0.0037687286, -0.0037229455, -0.0038502067, 0.00017585466, 0.0022807976, -0.005396993, 0.0025640226, -0.00059983414, 0.005108747, -0.00020239112, 0.00046563311, -0.0033968012, 0.0045874673, 0.008622

In [13]:
# X_train 구조 수정
X_train = [sublist[0] + sublist[1] + sublist[2] for sublist in X_train]

# X_test 구조 수정
X_test = [sublist[0] + sublist[1] + sublist[2] for sublist in X_test]

In [14]:
# 학습 데이터 준비
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_emotion_train = torch.FloatTensor(y_emotion_train)
y_emotion_test = torch.FloatTensor(y_emotion_test)
y_situation_train = torch.FloatTensor(y_situation_train)
y_situation_test = torch.FloatTensor(y_situation_test)

In [15]:
print(X_train)
print(X_test)
print(y_emotion_train)
print(y_emotion_test)
print(y_situation_train)
print(y_situation_test)

tensor([[ 9.9000e+01,  4.9000e+01, -7.6908e-03,  ..., -1.3143e-03,
         -8.5944e-04, -1.2682e-03],
        [ 1.0400e+02,  4.7000e+01, -5.5893e-03,  ..., -3.1088e-03,
         -1.2677e-03, -2.6877e-03],
        [ 1.0600e+02,  5.4000e+01,  6.5832e-04,  ..., -5.3649e-03,
         -2.7176e-03, -1.2393e-03],
        ...,
        [ 1.8400e+02,  8.9000e+01,  9.6485e-03,  ...,  8.8828e-03,
         -5.7029e-03,  5.0528e-03],
        [ 1.2000e+02,  6.3000e+01, -2.1890e-03,  ...,  2.8059e-03,
          6.4482e-03, -2.1146e-03],
        [ 1.3100e+02,  6.2000e+01,  7.6966e-03,  ...,  8.2683e-03,
         -6.0985e-03,  9.4383e-03]])
tensor([[ 8.4000e+01,  4.2000e+01,  5.7208e-03,  ...,  9.1750e-04,
          4.3007e-03,  8.5022e-04],
        [ 9.0000e+01,  4.6000e+01, -5.3623e-04,  ..., -7.0416e-03,
          9.0146e-04,  6.3925e-03],
        [ 1.0800e+02,  5.4000e+01, -8.6197e-03,  ..., -2.3915e-03,
         -9.5101e-03,  4.5059e-03],
        ...,
        [ 8.5000e+01,  4.3000e+01,  8.1323e-03

In [16]:
data_train = []
data_test = []

# 그래프 데이터 생성
for i in range(len(X_train)):
    edge_index = torch.tensor([[0, 1], [1, 0]], dtype=torch.long)  # 예시 엣지 (0-1)
    data = Data(x=X_train, edge_index=edge_index)  # 감정 및 상황 레이블은 아직 할당하지 않음
    data.y_emotion = y_emotion_train[i]  # 감정 레이블 할당
    data.y_situation = y_situation_train[i]  # 상황 레이블 할당
    data_train.append(data)

for i in range(len(X_test)):
    edge_index = torch.tensor([[0, 1], [1, 0]], dtype=torch.long)  # 예시 엣지 (0-1)
    data = Data(x=X_test, edge_index=edge_index)  # 감정 및 상황 레이블은 아직 할당하지 않음
    data.y_emotion = y_emotion_test[i]  # 감정 레이블 할당
    data.y_situation = y_situation_test[i]  # 상황 레이블 할당
    data_test.append(data)

In [17]:
print(data_train[861])
print(data_test[861])
print(data_train[862])
print(data_test[862])

Data(x=[41304, 102], edge_index=[2, 2], y_emotion=0.0, y_situation=0.0)
Data(x=[10326, 102], edge_index=[2, 2], y_emotion=0.0, y_situation=3.0)
Data(x=[41304, 102], edge_index=[2, 2], y_emotion=3.0, y_situation=3.0)
Data(x=[10326, 102], edge_index=[2, 2], y_emotion=5.0, y_situation=4.0)


In [18]:
class GNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GNNModel, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.conv3 = GCNConv(hidden_dim, hidden_dim)
        self.fc_emotion = nn.Linear(hidden_dim, output_dim['emotion'])
        self.fc_situation = nn.Linear(hidden_dim, output_dim['situation'])

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = self.conv3(x, edge_index)
        x = F.relu(x)
        x = x.mean(dim=0)  # 그래프의 특성을 하나의 벡터로 요약
        emotion_out = self.fc_emotion(x)
        situation_out = self.fc_situation(x)

        return emotion_out, situation_out

In [ ]:
# class GNNModel(nn.Module):
#     def __init__(self, input_dim, hidden_dim, output_dim):
#         super(GNNModel, self).__init__()
#         self.conv1 = SAGEConv(input_dim, hidden_dim)
#         self.conv2 = SAGEConv(hidden_dim, hidden_dim)
#         self.fc_emotion = nn.Linear(hidden_dim, output_dim['emotion'])
#         self.fc_situation = nn.Linear(hidden_dim, output_dim['situation'])

#     def forward(self, x, edge_index):
#         x = self.conv1(x, edge_index)
#         x = F.relu(x)
#         x = self.conv2(x, edge_index)
#         x = F.relu(x)
#         x = x.mean(dim=0)  # 그래프의 특성을 하나의 벡터로 요약
#         emotion_out = self.fc_emotion(x)
#         situation_out = self.fc_situation(x)

#         return emotion_out, situation_out


In [19]:
# 학습 모델
def train(model, optimizer, data_loader, task):
    model.train()  # Set the model to train mode
    total_loss = 0

    for data in data_loader:
        optimizer.zero_grad()  # Initialize gradients

        x, edge_index, labels = data.x, data.edge_index, data.y
        if task == 'emotion':
            labels = data.y_emotion
        elif task == 'situation':
            labels = data.y_situation

        out_emotion, out_situation = model(x, edge_index)  # Separate outputs for emotion and situation tasks

        if task == 'emotion':
            out = out_emotion
        elif task == 'situation':
            out = out_situation

        # Compute the loss function
        loss = F.cross_entropy(out, labels)

        # Backpropagation and weight updates
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * x.size(0)

    return total_loss / len(data_loader.dataset)

In [20]:
# 평가 함수
def evaluate(model, data_loader, task):
    model.eval()  # 모델을 평가 모드로 설정
    total_correct = 0
    total_f1 = 0

    with torch.no_grad():
        for data in data_loader:

            # 데이터 배치에서 입력과 정답을 가져옴
            x, edge_index, labels = data.x, data.edge_index, data.y
            if task == 'emotion':
                labels = data.y_emotion
                max_sentence_length  = 6
            elif task == 'situation':
                labels = data.y_situation
                max_sentence_length = 12
                            
            labels = torch.nn.functional.pad(labels, (0, max_sentence_length - labels.shape[0]), value=-1)

            # 모델의 출력 계산
            out_emotion, out_situation = model(x, edge_index)

            if task == 'emotion':
                out = out_emotion
            elif task == 'situation':
                out = out_situation
            
            # 정확도 계산
            _, pred = torch.max(out.unsqueeze(1), dim=1)
            correct = pred.eq(labels).sum().item()
            total_correct += correct

            # F1 점수 계산
            f1 = f1_score(labels.cpu().numpy(), pred.cpu().numpy(), average='macro')

            total_f1 += f1

    accuracy = total_correct / len(data_loader.dataset)
    f1_result = total_f1 / len(data_loader.dataset)

    return accuracy, f1_result

In [21]:
# 데이터 로더 설정
emotion_batch_size = 6
situation_batch_size = 12

emotion_train_dataset = data_train
situation_train_dataset = data_train
emotion_test_dataset = data_test
situation_test_dataset = data_test

emotion_train_loader = DataLoader(emotion_train_dataset, batch_size=emotion_batch_size, shuffle=True)
situation_train_loader = DataLoader(situation_train_dataset, batch_size=situation_batch_size, shuffle=True)
emotion_test_loader = DataLoader(emotion_test_dataset, batch_size=emotion_batch_size, shuffle=False)
situation_test_loader = DataLoader(situation_test_dataset, batch_size=situation_batch_size, shuffle=False)

In [22]:
# 모델 훈련
num_epochs = 1

# 모델 초기화
input_dim = 102  # 입력 특성의 차원
hidden_dim = 32  # 은닉 상태의 차원
output_dim = {'emotion': 6, 'situation': 12}  # 출력의 차원 (감정: 6개 클래스, 상황: 12개 클래스)

learning_rates = [0.001, 0.01, 0.1]
weight_decays = [1e-4, 5e-4, 1e-3]

best_accuracy = 0.0
best_f1 = 0.0
best_learning_rate = None
best_weight_decay = None

for lr in learning_rates:
    for wd in weight_decays:
        # 모델 초기화
        model = GNNModel(input_dim, hidden_dim, output_dim)

        # 최적화 설정
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=wd)

        # 모델 훈련
        for epoch in range(num_epochs):
            # 감정 학습
            model.train()
            emotion_train_loss = train(model, optimizer, emotion_train_loader, task='emotion')

            # 감정 평가
            emotion_acc, emotion_f1 = evaluate(model, emotion_test_loader, task='emotion')

            # 상황 학습
            model.train()
            situation_train_loss = train(model, optimizer, situation_train_loader, task='situation')

            # 상황 평가
            situation_acc, situation_f1 = evaluate(model, situation_test_loader, task='situation')

            # 결과 출력
            print(f"Epoch [{epoch+1}/{num_epochs}]")
            print(f"Emotion - Train Loss: {emotion_train_loss:.4f}, Acc: {emotion_acc:.4f}, F1: {emotion_f1:.4f}")
            print(f"Situation - Train Loss: {situation_train_loss:.4f}, Acc: {situation_acc:.4f}, F1: {situation_f1:.4f}")
            print("--------------------------------------------------")

        # 최고 성능인 경우 기록
        if emotion_acc > best_accuracy:
            best_accuracy = emotion_acc
            best_f1 = emotion_f1
            best_learning_rate = lr
            best_weight_decay = wd

print("Grid Search Results:")
print(f"Best Learning Rate: {best_learning_rate}")
print(f"Best Weight Decay: {best_weight_decay}")
print(f"Best Emotion Accuracy: {best_accuracy}")
print(f"Best Emotion F1 Score: {best_f1}")

Epoch [1/1]
Emotion - Train Loss: 1180650.3005, Acc: 0.1168, F1: 0.0078
Situation - Train Loss: 6221063.8389, Acc: 0.0800, F1: 0.0016
--------------------------------------------------
Epoch [1/1]
Emotion - Train Loss: 1181397.7636, Acc: 0.1168, F1: 0.0078
Situation - Train Loss: 6221039.9090, Acc: 0.0800, F1: 0.0016
--------------------------------------------------
Epoch [1/1]
Emotion - Train Loss: 1180989.2145, Acc: 0.1168, F1: 0.0078
Situation - Train Loss: 6222158.4272, Acc: 0.0800, F1: 0.0016
--------------------------------------------------
Epoch [1/1]
Emotion - Train Loss: 1179772.5387, Acc: 0.1168, F1: 0.0078
Situation - Train Loss: 6224101.3242, Acc: 0.0800, F1: 0.0016
--------------------------------------------------
Epoch [1/1]
Emotion - Train Loss: 1180146.6174, Acc: 0.1168, F1: 0.0078
Situation - Train Loss: 6223838.1940, Acc: 0.0800, F1: 0.0016
--------------------------------------------------
Epoch [1/1]
Emotion - Train Loss: 1180352.9109, Acc: 0.1168, F1: 0.0078
Sit